In [15]:
#imports and authentications

import spotipy
import pandas as pd
import datetime
import time
import os
from json.decoder import JSONDecodeError
import numpy as np
import pandas as pd

# music api imports
import pygn
import traceback
import spotipy
import spotipy.util as util

import requests
import re
from time import sleep
import pickle

import matplotlib.pyplot as plt
#import seaborn as sns
%matplotlib inline

scope = 'user-library-read'
username = "karishma.gulati"
client_id="afe7c584ab704e5487b31b52a9d0009c"
client_secret="fa08966efd794dbb88a484e9a54b11e1"
redirect_uri="http://localhost:8888/"

# token = util.prompt_for_user_token(<INSERT SPOTIFY USERNAME>,
#                                   client_id=<INSERT SPOTIFY CLIENT ID>,
#                                   client_secret=<INSERT SPOTIFY CLIENT SECRET>,
#                                   redirect_uri=<INSERT REDIRECT URI>)

try:
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)
except (AttributeError, JSONDecodeError):
    os.remove(f".cache-{username}")
    token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

spotify = spotipy.Spotify(auth=token)

In [16]:
#defining functions

# get categories
def get_categories():
    category_ids = []
    for i in spotify.categories(limit = 50)['categories']['items']:
        category_ids.append(i.get('id'))
    return category_ids

# get playlists from list of categories
def get_playlists(categories):
    playlist_ids = []
    for i in categories:
        for j in spotify.category_playlists(i, limit = 50)['playlists']['items']:
            playlist_ids.append(j.get('id'))
    return playlist_ids

# get song ids from list of playlist ids
def get_songs(playlists):
    song_ids = []
    count = 0
    for i in playlists:
        try:
            for j in spotify.user_playlist('spotify', i)['tracks']['items']:
                song_ids.append([j['track']['id'], j['track']['name'], j['track']['artists'][0]['name']])
        except:
            pass
        time.sleep(.1)
    return song_ids

def chunks(l, n):
    n = max(1, n)
    return [l[i:i+n] for i in range(0, len(l), n)]

def complete_audio_features(songs_ids):
    features_list = []
    for j in chunks(list(songs_ids),50):
        features_add = spotify.audio_features(tracks=j)
        features_list.extend(features_add)
    return features_list

def make_audio_features_df(audio_features):
    audio_features_df = pd.DataFrame(columns = audio_features[0])
    for i in range(0,len(audio_features)):
        audio_features_df = audio_features_df.append(audio_features[i], ignore_index=True)
    return audio_features_df

In [17]:
# Getting Song Information for these song IDs

In [18]:
print("The spotify token has been fetched!\n")
categories = get_categories()
playlists = get_playlists(categories)
songs = get_songs(playlists)
songs_df = pd.DataFrame(songs)
songs_df.head(10)

The spotify token has been fetched!



,0,1,2
0,3YU6vJbjYUG0tiJyXf9x5V,Him & I (with Halsey),G-Eazy
1,1bhUWB0zJMIKr9yVPrkEuI,Perfect Duet (Ed Sheeran & Beyoncé),Ed Sheeran
2,0OI7AFifLSoGzpb8bdBLLV,"Home (with Machine Gun Kelly, X Ambassadors & ...",Machine Gun Kelly
3,2amzBJRBPOGszBem4FedfE,Let You Down,NF
4,42CeaId2XNlxugDvyqHfDf,Candy Paint,Post Malone
5,2xmrfQpmS2iJExTlklLoAL,I Miss You (feat. Julia Michaels),Clean Bandit
6,0tBbt8CrmxbjRP0pueQkyU,Wolves,Selena Gomez
7,7eFmN6wnsb7WowRKAqRFfs,Wait,Maroon 5
8,2z4pcBLQXF2BXKFvd0BuB6,Tip Toe (feat. French Montana),Jason Derulo
9,3al2hpm92xE0pBalqWQHdD,Betrayed,Lil Xan


In [19]:
songs_df.columns = ['id', 'name', 'artist']
songs_df = songs_df.drop_duplicates()
print("Number of unique songs: ", len(songs_df.index))

Number of unique songs:  36016


In [ ]:
audio_features = complete_audio_features(songs_df['id'])
audio_features_df = make_audio_features_df(audio_features)
audio_features_df = audio_features_df.drop_duplicates()
print("Number of unique songs post receiving audio features: ", len(audio_features_df.index))

In [ ]:
songs_final_df = songs_df.merge(audio_features_df, how='inner', on='id')
del songs_final_df['track_href']
del songs_final_df['analysis_url']
del songs_final_df['uri']
del songs_final_df['type']

songs_final_df.head(10)

In [ ]:
print("Final song list has songs: ", len(songs_final_df.index))

In [ ]:
#songs_final_df.to_csv("all_songs.csv", index=False)

In [ ]:
#Merge all songs with features from Gracenote

In [ ]:
# timing function
def timefunc(f):
    def f_timer(*args, **kwargs):
        start = time.time()
        result = f(*args, **kwargs)
        end = time.time()
        print(f.__name__, 'took', end - start, 'seconds')
        return result
    return f_timer

In [ ]:
clientid='1143049347-DB6E8C78665AF7AE447BE4903B6F2BF5'
userid = pygn.register(clientid)

In [ ]:
songs = pd.read_csv('all_songs.csv')
songs.head(10)

In [ ]:
# Get track info from gracenote
# NOTE: Gracenote API has built-in fuzzy matching to artist and track.
from collections import defaultdict

def get_gn_multiple(search, dictionary, item):
    '''
    Helper function to get multiple items within Gracenote record
    '''
    for i in search[item].items():
        dictionary[item + '_' + i[0]] = i[1]['TEXT']

def get_gn(artist, track):
    '''
    Gets artist and track information from Gracenote
    '''
    gn_dict = defaultdict(list)
    gn_info = pygn.search(clientid, userid, artist=artist, track=track)
    
    print(artist,track)
    print(gn_info)
    gn_dict['gnid'] = gn_info['track_gnid']

    # artist specific info
    for a in ['artist_origin', 'artist_type', 'artist_era']:
        get_gn_multiple(gn_info, gn_dict, a)
    # track specific info
    for s in ['genre', 'mood', 'tempo']: # can potentially drop 'tempo' since Spotify has already captured this
        get_gn_multiple(gn_info, gn_dict, s)      
    return dict(gn_dict)

@timefunc
def getDetailsFromGracenote(df):
    d = {}
    i = 0
    for index, row in df.iterrows():
        try:
            x = get_gn(row['artist'], row['name'])
            d[row['id']] = x
        except:
            print(row)
            traceback.print_exc()
            print(i)
            i = i+1
            if(i == 20):
                sys.exit(1)
    gn_df = pd.DataFrame(d).transpose().reset_index()
    gn_df = gn_df.rename(columns = {'index' : 'id'})
    return gn_df

In [18]:
final_gn_df = getDetailsFromGracenote(songs)
final_gn_df

songs_final_df = songs.merge(final_gn_df, how='inner', on='id')
print("Final song list without eliminating duplicates: ", len(songs_final_df.index))

songs_final_df = songs_final_df.drop_duplicates()
print("Final song list after eliminating duplicates: ", len(songs_final_df.index))

#songs_final_df.to_csv("all_songs_with_labels.csv", index=False)

5561

In [19]:
all_songs = pd.read_csv("all_songs_with_labels.csv")
all_songs = all_songs[pd.notnull(all_songs['mood_1'])]
all_songs.mood_1.unique()
print("Songs without NaN label values", len(all_songs.index))

all_songs = all_songs[all_songs.mood_1 != 'Other']
print("Songs without Other label values", len(all_songs.index))
all_songs.mood_1.unique()

#Excited = happy
#Sensual = Relaxed
#Rowdy = Angry
#Cool = Relaxed
#Defiant = Angry
#Energizing = happy
#Urgent = Angry
#Sophisticated = Relaxed
#Yearning = Sad
#Melancholy = Sad
#Brooding = Sad
#Lively = happy
#Empowering = happy
#Aggressive = angry
#Upbeat = Happy
#Somber = Sad
#Peaceful = Relaxed
#Gritty = Sad
#Fiery = Angry
#Easygoing = Relaxed
#Romantic = happy
#Sentimental = sad
#Stirring = sad
#Tender = relaxed
#Serious = sad

all_songs.mood_1[all_songs.mood_1 == 'Excited'] = 'Happy' 
all_songs.mood_1[all_songs.mood_1 == 'Sensual'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Rowdy'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Cool'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Defiant'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Energizing'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Urgent'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Sophisticated'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Yearning'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Melancholy'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Brooding'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Lively'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Empowering'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Aggressive'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Upbeat'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Somber'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Peaceful'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Gritty'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Fiery'] = 'Angry'
all_songs.mood_1[all_songs.mood_1 == 'Easygoing'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Romantic'] = 'Happy'
all_songs.mood_1[all_songs.mood_1 == 'Sentimental'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Stirring'] = 'Sad'
all_songs.mood_1[all_songs.mood_1 == 'Tender'] = 'Relaxed'
all_songs.mood_1[all_songs.mood_1 == 'Serious'] = 'Sad'

all_songs.mood_1.unique()

Songs without NaN label values 4510
Songs without Other label values 4446


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/

array(['Happy', 'Relaxed', 'Angry', 'Sad'], dtype=object)

In [21]:
#all_songs.to_csv("all_songs_with_modified_labels.csv", index=False)